# ESC Summer 1주차 과제

7조 곽현지

### Q1) 이 경우에는 type1 error가 높을까요? type2 error가 높을까요?

이 경우 threshold가 0.3으로 낮기 때문에, FP는 높고 FN은 낮다. FN은 type1 error, FP는 type2 error에 대응하기 떄문에 treshold가 낮은 경우애는 type2 erro가 type1 error보다 높다.

### Q2-1) Accuracy, Precision and Recall이  무엇인지 TP, FP, FN, TN의 식으로 나타내 주세요.

Accuracy = (TP + TN) / (TP + FP + FN + TN)  :맞게 예측한 비율  
Precision = TP / (TP + FP)  :모델이 TRUE로 예측한 것 중, 실제 TRUE의 비율  
Recall = TP / (TP + TN)  :실제 TRUE인 것 중, 모델이 TRUE로 예측한 비율  

### Q2-2) Precision and Recall에 관한 예시를 들어주세요. 생각한 예시에서 threshold를 높인다는 것은 무슨 의미인가요? 

은행이 고객의 대출가능여부를 판단할때에는 threshold를 높이는 것이 합리적이다. 은행의 입장에서 고객이 대출금을 상환하지 않는 것은 영업적 손해에 해당하기 때문에 이를 방지하기 위해서는 threshold를 높여 대출심사를 까다롭게 수행하는 것이 힙리적이다.

### Q3) 코드 따라해보고 주석 달기

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# MNIST Dataset
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  #Train = True면 trainset
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 

# Hyper Parameters  
input_size = 784  #28*28
num_classes = 10  #0~9
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# Dataset Loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [3]:
# Defining model
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [4]:
model = LogisticRegression(input_size, num_classes) 

In [5]:
# loss function(classification problem -> crossentrophy)
criterion = nn.CrossEntropyLoss() 
# optimizer(SGD)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [6]:
# Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # reset all gradients to 0
        outputs = model(images) # forward propagation
        loss = criterion(outputs, labels) # calculate the loss
        loss.backward() # backward propagation
        optimizer.step() # updating weights
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data.item()))
        
# loss.data[0] 작동 안됨. loss.data(tensor형태) or loss.data.item()(상수형태) 사용

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2367
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1012
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0236
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9473
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8521
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.8009
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7530
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6610
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5721
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5947
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5055
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4754
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3811
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4604
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.4158
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2610
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2618
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2896
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2621
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1452
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [7]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) # 주어진 텐서 배열의 최대값이 들어있는 index 리턴
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % (100* correct/total)) 

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Accuracy of the model on the 10000 test images:  83 %


### Q4-1) 2-4 코드에서 optim.SDG를 사용하지 않고 코드를 짠다면 어떤 방식으로 짜야할까요? 

training을 위한 for문에, weight(bias)를 update하는 w.data = w.data - learningrate * w.grad.data와  
초기화하는 w.grad.data.zero_() 구문을 추가한다. 

In [8]:
# data
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 

# initiaize weights
w = Variable(torch.Tensor([1.0]), requires_grad = True)
b = Variable(torch.Tensor([1.0]), requires_grad = True)

def forward(x):
    return x * w + b

def cost(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

# training data
nb_epochs = 1000 
for epoch in range(nb_epochs + 1):
    for x_val, y_val in zip(x_train, y_train):
        c = cost(x_val, y_val)
        c.backward()
        w.data = w.data - 0.01 * w.grad.data
        b.data = b.data - 0.01 * b.grad.data
        
        # weight update 후 gradient 초기화 
        w.grad.data.zero_()
        b.grad.data.zero_()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w.item(), b.item(), c.item()
        ))

Epoch    0/1000 W: 0.895, b: 0.945 Cost: 0.621575
Epoch  100/1000 W: 0.834, b: 0.394 Cost: 0.016864
Epoch  200/1000 W: 0.922, b: 0.186 Cost: 0.003754
Epoch  300/1000 W: 0.963, b: 0.088 Cost: 0.000836
Epoch  400/1000 W: 0.983, b: 0.041 Cost: 0.000186
Epoch  500/1000 W: 0.992, b: 0.020 Cost: 0.000041
Epoch  600/1000 W: 0.996, b: 0.009 Cost: 0.000009
Epoch  700/1000 W: 0.998, b: 0.004 Cost: 0.000002
Epoch  800/1000 W: 0.999, b: 0.002 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


### Q4-2) Gradient descent 코드 구현하는 문제입니다. 2-4 코드에서 다른 optimizer 3개를 이용하여 결과값을 비교해주세요.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
# For reproducibility
torch.manual_seed(1)

#### Adam

In [11]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 

# 모델 초기화
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True) 

# optimizer 설정
optimizer = optim.Adam([W, b], lr=0.01) 

nb_epochs = 1000 
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.010, b: 0.010 Cost: 4.666667
Epoch  100/1000 W: 0.660, b: 0.628 Cost: 0.081239
Epoch  200/1000 W: 0.733, b: 0.591 Cost: 0.050898
Epoch  300/1000 W: 0.774, b: 0.501 Cost: 0.036588
Epoch  400/1000 W: 0.815, b: 0.410 Cost: 0.024475
Epoch  500/1000 W: 0.854, b: 0.324 Cost: 0.015306
Epoch  600/1000 W: 0.888, b: 0.248 Cost: 0.008968
Epoch  700/1000 W: 0.917, b: 0.184 Cost: 0.004927
Epoch  800/1000 W: 0.941, b: 0.132 Cost: 0.002537
Epoch  900/1000 W: 0.959, b: 0.091 Cost: 0.001224
Epoch 1000/1000 W: 0.972, b: 0.061 Cost: 0.000552


#### ASGD

In [12]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 

# 모델 초기화
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True) 

# optimizer 설정
optimizer = optim.ASGD([W, b], lr=0.01) 

nb_epochs = 1000 
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007443
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004601
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002844
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001758
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001087
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000673
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000416
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000258
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000160


#### Adadelta

In [13]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 

# 모델 초기화
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True) 

# optimizer 설정
optimizer = optim.Adadelta([W, b], lr=0.01) 

nb_epochs = 1000 
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.000, b: 0.000 Cost: 4.666667
Epoch  100/1000 W: 0.004, b: 0.004 Cost: 4.612791
Epoch  200/1000 W: 0.009, b: 0.009 Cost: 4.545029
Epoch  300/1000 W: 0.015, b: 0.015 Cost: 4.466577
Epoch  400/1000 W: 0.022, b: 0.022 Cost: 4.379364
Epoch  500/1000 W: 0.029, b: 0.029 Cost: 4.284824
Epoch  600/1000 W: 0.037, b: 0.037 Cost: 4.184107
Epoch  700/1000 W: 0.046, b: 0.046 Cost: 4.078178
Epoch  800/1000 W: 0.055, b: 0.055 Cost: 3.967873
Epoch  900/1000 W: 0.064, b: 0.064 Cost: 3.853932
Epoch 1000/1000 W: 0.074, b: 0.074 Cost: 3.737018
